In [29]:
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split


from datetime import date, timedelta
import yfinance as yf #Alternative package if webreader does not work: pip install yfinance
import numpy as np # Fundamental package for scientific computing with Python
import joblib
from lib.Measures import measureDataframe2
from lib.Utility import exportExcelWithTimeStamp, getData, computeEquity
import plotly.express as px
from plotly import graph_objects as go
# Train the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

commissioni = 0.0005
threshold = 0.5
features2 = ['feat_Open-PrevClose']
features = ['feat_Open-PrevClose', 'feat_vixOpen', 'feat_vixPrevClose', 'feat_vvixOpen', 'feat_vvixPrevClose']
bigbodyRangeValue = 0.0085

In [30]:
end = date.today() - timedelta(30)
start = "1990-09-04"

data = getData(start, end)

print(data)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

           Date   Open   High    Low  Close  Adj Close  Volume  feat_spxGap  \
4118 2007-01-04  12.40  12.42  11.28  11.51      11.51       0     0.000000   
4119 2007-01-05  11.84  12.25  11.68  12.14      12.14       0     0.000000   
4120 2007-01-08  12.48  12.83  11.78  12.00      12.00       0    -0.000319   
4121 2007-01-09  11.86  12.47  11.69  11.91      11.91       0     0.000000   
4122 2007-01-10  12.34  12.50  11.43  11.47      11.47       0    -0.002415   
...         ...    ...    ...    ...    ...        ...     ...          ...   
8518 2024-06-28  12.24  12.76  11.87  12.44      12.44       0     0.001023   
8519 2024-07-01  12.98  13.26  12.10  12.22      12.22       0     0.001941   
8520 2024-07-02  12.67  12.88  11.85  12.03      12.03       0    -0.002420   
8521 2024-07-03  12.13  12.23  11.95  12.09      12.09       0    -0.000285   
8522 2024-07-05  12.37  12.61  11.84  12.48      12.48       0     0.000161   

      feat_spxPrevRange  feat_Spx_Open-PrevClose  .

In [31]:
print(features)

x_train, x_test, y_train, y_test = train_test_split(data[features], data['Cluster'], test_size= 0.3, random_state= 1,shuffle=True)

x_train = x_train.sort_index()
x_test = x_test.sort_index()
y_train= y_train.sort_index()
y_test = y_test.sort_index()

print (f'X_train: {x_train.shape} \nX_test: {x_test.shape} \ny_train: {y_train.shape} \ny_test: {y_test.shape}')

['feat_Open-PrevClose', 'feat_vixOpen', 'feat_vixPrevClose', 'feat_vvixOpen', 'feat_vvixPrevClose']
X_train: (3077, 5) 
X_test: (1319, 5) 
y_train: (3077,) 
y_test: (1319,)


In [32]:
# Set regularization rate
reg = 0.01

# train a logistic regression model on the training set
model = LogisticRegression(C=1/reg, solver="liblinear", random_state=42).fit(x_train, y_train)

joblib.dump(model, "./VIXkONG.joblib")

['./VIXkONG.joblib']

In [33]:
df = x_test.copy()

df['predictions'] = model.predict(x_test.sort_index())
df['y_test'] = y_test

print(f"Accuracy score: {accuracy_score(df['y_test'], df['predictions'])}")

Accuracy score: 0.6595905989385898


In [34]:

# Seleziona le colonne con il prefisso "feat"
feat_cols = [col for col in data.columns if 'feat' in col]

test_data = data[data.index.isin(x_test.index)]

test_data['cluster'] = model.predict(x_test)

text_pred_prob = model.predict_proba(x_test)

df_test = computeEquity(test_data, commissioni, feat_cols, text_pred_prob)
df_test['color'] = np.where(test_data['cluster']==1,'green','red')

measureDataframe2(df_test)
exportExcelWithTimeStamp(df_test, "Equity/", "TestEquity.xlsx")



fig = go.Figure(data=[go.Candlestick(x=test_data['Date'],
                open=test_data['Open'],
                close=test_data['Close'], high=test_data['High'], low=test_data['Low'], opacity=0.5)])
sec_y = go.layout.YAxis(title='Equity')#, offset=dict(type="absolute", value=0.25))
fig.add_trace(go.Scatter(x=df_test['Date'], y=df_test['equity'], name='Equity', line=dict(color='blue')) )
fig.add_scatter(x=test_data['Date'], y=test_data['Open'], mode='markers',marker=dict(color=df_test['color']))

fig.update_layout(
    xaxis_rangeslider_visible=False,
                  yaxis2=dict(
        title='Equity',
        overlaying='y',  # Sovrappone il secondo asse y al primo
        side='right',  # Posiziona il secondo asse y a destra
        showgrid=False  # Nasconde la griglia per il secondo asse y se necessario
    ))
# Associa la serie Equity al secondo asse y
fig.data[1].update(yaxis='y2')
fig.show()

# Plot distribution of values in Marks column
# df_test['gain'].plot(kind='kde')
# plt.grid()

C:\Users\vitto\AppData\Local\Temp\ipykernel_23292\2487315734.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Equity: 529.5167721271512
Misure per 5.495833333333334 anni
Benchmark Equity: -131.85992622375488
Equity 529.5167721271512
Maximum number of loss operation: 6
Maximum DrawDown: -18.131693052291894
SGR: 96.34876824148316 vs Benchmark: -23.992708334875793
Win Rate: 65.73161485974222
Long Equity 205.4200029373169
Short Equity 337.2799291610718
Numero di Eseguiti 1319


In [35]:
before = date.today() - timedelta(400)
tomorrow = date.today() + timedelta(1)

dataLive = getData(before, tomorrow)



dataLive =dataLive.dropna()
dataLive.head(15)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume,feat_spxGap,feat_spxPrevRange,feat_Spx_Open-PrevClose,...,feat_vixPrevClose,feat_vvixOpen,feat_vvixPrevClose,feat_vvixPrevOpen,feat_vvixPrevHigh,feat_vvixPrevLow,feat_vvixPrevHigh-Low,feat_Acc,Cluster,feat_PrevCluster
1,2023-07-05,14.19,14.740000,14.05,14.18,14.18,0,-0.003041,0.003184,0,...,13.57,88.309998,84.169998,85.709999,85.739998,83.709999,0.023685,0,-1,-1
2,2023-07-06,14.85,17.080000,14.79,15.44,15.44,0,-0.005442,0.003928,0,...,14.18,93.750000,87.760002,88.309998,88.839996,86.940002,0.021515,1,1,-1
3,2023-07-07,15.97,16.059999,14.33,14.83,14.83,0,-0.001598,0.008495,0,...,15.44,97.830002,95.300003,93.750000,104.750000,93.739998,0.117440,1,-1,1
4,2023-07-10,16.08,16.209999,15.04,15.07,15.07,0,-0.001073,0.009760,0,...,14.83,95.599998,94.489998,97.830002,97.830002,92.230003,0.057242,0,-1,-1
5,2023-07-11,15.02,15.250000,14.63,14.84,14.84,0,0.001365,0.005161,1,...,15.07,94.970001,95.330002,95.599998,97.300003,94.739998,0.026778,1,-1,-1
6,2023-07-12,14.82,14.820000,13.51,13.54,13.54,0,0.006404,0.007967,1,...,14.84,94.660004,95.209999,94.970001,95.879997,93.959999,0.020217,0,-1,-1
7,2023-07-13,13.44,13.610000,13.12,13.61,13.61,0,0.004324,0.005620,1,...,13.54,94.849998,95.849998,94.660004,97.330002,91.330002,0.063385,1,1,-1
8,2023-07-14,13.72,13.760000,13.22,13.34,13.34,0,0.001013,0.006238,1,...,13.61,96.220001,96.430000,94.849998,97.699997,94.230003,0.036584,1,-1,1
9,2023-07-17,13.78,14.000000,13.43,13.48,13.48,0,0.000764,0.006246,1,...,13.34,96.570000,94.889999,96.220001,98.050003,94.800003,0.033777,0,-1,-1
10,2023-07-18,13.61,13.670000,13.29,13.30,13.30,0,-0.000223,0.006199,0,...,13.48,96.070000,95.980003,96.570000,96.599998,93.629997,0.030755,1,-1,-1


In [37]:
# CLASSIFICATION
clf = joblib.load("./VIXkONG.joblib")

# Seleziona le colonne con il prefisso "feat"
# features = [col for col in data.columns if 'feat' in col]
# features = ['feat_Open', 'feat_vixOpen', 'feat_vixPrevClose', 'feat_vvixOpen', 'feat_Acc', 'feat_bigBodyRange']
print(features)

x_live = dataLive[features]

y_live_pred = clf.predict(x_live)
dataLive['cluster'] = y_live_pred

print(f"Accuracy score: {accuracy_score(dataLive['Cluster'], dataLive['cluster'])}")

# Seleziona le colonne con il prefisso "feat"
# feat_cols = [col for col in dataLive.columns if 'feat' in col]

dataLive = dataLive[dataLive.index.isin(x_live.index)]

live_pred_prob = clf.predict_proba(x_live)

df_live = computeEquity(dataLive, commissioni, feat_cols, live_pred_prob)
df_live['color'] = np.where(dataLive['cluster']==1,'green','red')

measureDataframe2(df_live)

exportExcelWithTimeStamp(df_live, "Equity/", "LiveEquity.xlsx")

# LIVE

fig = go.Figure(data=[go.Candlestick(x=df_live['Date'],
                open=df_live['Open'],
                close=df_live['Close'], high=df_live['High'], low=df_live['Low'], opacity=0.5)])
# sec_y = go.layout.YAxis(title='Equity')#, offset=dict(type="absolute", value=0.25))

# fig.add_trace(go.Line(x=df_live['Date'], y=df_live['equity'], name='Equity'))
fig.add_trace(go.Scatter(x=df_live['Date'], y=df_live['equity'], name='Equity', line=dict(color='blue')) )
fig.add_scatter(x=df_live['Date'], y=df_live['Open'], mode='markers',marker=dict(color=df_live['color']))

fig.update_layout(
    xaxis_rangeslider_visible=False,
                  yaxis2=dict(
        title='Equity',
        overlaying='y',  # Sovrappone il secondo asse y al primo
        side='right',  # Posiziona il secondo asse y a destra
        showgrid=False  # Nasconde la griglia per il secondo asse y se necessario
    ))
# Associa la serie Equity al secondo asse y
fig.data[1].update(yaxis='y2')
fig.show()

['feat_Open-PrevClose', 'feat_vixOpen', 'feat_vixPrevClose', 'feat_vvixOpen', 'feat_vvixPrevClose']
Accuracy score: 0.6459854014598541
Equity: 42.958658355236054
Misure per 1.1416666666666666 anni
Benchmark Equity: -23.700008392333984
Equity 42.958658355236054
Maximum number of loss operation: 5
Maximum DrawDown: -15.515002899169922
SGR: 37.62802191699509 vs Benchmark: -20.759131438540717
Win Rate: 63.86861313868613
Long Equity 10.639997482299805
Short Equity 34.34000587463379
Numero di Eseguiti 274
